# Integration Tests for Fast Withdrawal Indexer

In [1]:
from docs.scenarios.setup import setup
from scripts.defaults import (
    FAST_WITHDRAWAL_CONTRACT,
    INDEXER_GRAPHQL_URL,
)
from gql import Client
from scripts.helpers.contracts.fast_withdrawal import FastWithdrawal
from scripts.helpers.addressable import get_address
from docs.scenarios.indexer_integration import (
    IndexerTestEnvironment,
    random_pkh,
    fast_withdrawal_bridge_operation_query,
    create_withdrawal_from_l2_transaction,
    request_bridge_operation_with_high_verbosity,
)
from gql.transport.aiohttp import AIOHTTPTransport
from random import randint

web3, etherlink_account, tezos_account = setup()
fast_withdrawal = FastWithdrawal.from_address(tezos_account, FAST_WITHDRAWAL_CONTRACT)
transport = AIOHTTPTransport(url=INDEXER_GRAPHQL_URL)
indexer = Client(transport=transport, fetch_schema_from_transport=True)

test_env = IndexerTestEnvironment(
    fast_withdrawal=fast_withdrawal,
    provider=tezos_account,
    l2_caller=etherlink_account,
    withdrawer_pkh=random_pkh(tezos_account),
    indexer=indexer,
    discount_rate=0.99,
)

Setup:
- Tezos account: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`, balance: `100.048352 ꜩ`
- Etherlink account: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`, balance: `90.611787701 ꜩ`


### Test: User withdrawal creation and indexing

In [2]:
full_amount = randint(1000, 10_000)
l2_hash = test_env.make_xtz_withdrawal(full_amount)

Making Fast Withdrawal, XTZ:
  - Sender: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `http://etherlink.dipdup.net`
  - Withdrawal params:
      * Target: `tz1WiZDC7NgfVxaySgqD23C17KvZNc442rMi`
      * Fast Withdrawal contract: `KT1AR4mD3n9JxD7yB4RcYUybyeq9BBi9W2fm`
      * Payload bytes: `0500878c01`
      * Amount (mutez): `9_058`
      * Discounted amt (mutez): `8_967`
      * Fee (mutez): `91`
Successfully initiated XTZ Fast Withdrawal, tx hash: `0xbe9816c5b30946c73d5a62ef7a78c36940120e90b9d8b47c4613ba4dc0f35e60`


- [x] Verify that a user-initiated withdrawal is indexed with the status `CREATED` and kind `fast_withdrawal`
- [x] Measure the time between the withdrawal's creation on `L2` and the moment it is indexed

In [3]:
bridge_operation = await request_bridge_operation_with_high_verbosity(
    test_env=test_env,
    l2_hash=l2_hash,
    kind='fast_withdrawal',
    status='CREATED',
)

l2_transaction = bridge_operation['withdrawal']['l2_transaction']
assert l2_transaction['amount'] == str(full_amount) + '0'*12
assert l2_transaction['l1_account'] == test_env.withdrawer_pkh
assert l2_transaction['l2_account'] == test_env.l2_caller.address.lower().split('0x')[1]
assert not bridge_operation['is_completed']
assert not bridge_operation['is_successful']
assert bridge_operation['l1_account'] == test_env.withdrawer_pkh
assert bridge_operation['l2_account'] == test_env.l2_caller.address.lower().split('0x')[1]
assert bridge_operation['withdrawal']['l1_transaction'] is None

Requesting bridge operation:
- l2_hash: `be9816c5b30946c73d5a62ef7a78c36940120e90b9d8b47c4613ba4dc0f35e60`
- kind: `fast_withdrawal`
- status: `CREATED`
- attempts: xxxxxx
Found withdrawal, id: `19`
Elapsed time (seconds): `12.4`


### Test: Correct and timely `payout_withdrawal` execution

In [4]:
withdrawal = create_withdrawal_from_l2_transaction(l2_transaction)
opg = test_env.make_payout_withdrawal(withdrawal)
result = tezos_account.wait(opg)

- [x] Verify that the service provider receives an indexer entry with status `CREATED` and kind `fast_withdrawal_service_provider`  
- [x] Verify that `bridge_operation.l1_account` is updated to the address of the service provider

In [5]:
bridge_operation = await request_bridge_operation_with_high_verbosity(
    test_env=test_env,
    l2_hash=l2_hash,
    kind='fast_withdrawal_service_provider',
    status='CREATED',
)

l2_transaction = bridge_operation['withdrawal']['l2_transaction']
assert l2_transaction['amount'] == str(full_amount) + '0'*12
assert l2_transaction['l1_account'] == test_env.withdrawer_pkh
assert l2_transaction['l2_account'] == test_env.l2_caller.address.lower().split('0x')[1]
assert not bridge_operation['is_completed']
assert not bridge_operation['is_successful']
assert bridge_operation['l1_account'] == get_address(test_env.provider), "should became provider's address"
assert bridge_operation['l2_account'] == test_env.l2_caller.address.lower().split('0x')[1]
assert bridge_operation['withdrawal']['l1_transaction'] is None
assert bridge_operation['withdrawal']['outbox_message']['builder'] == 'kernel'

Requesting bridge operation:
- l2_hash: `be9816c5b30946c73d5a62ef7a78c36940120e90b9d8b47c4613ba4dc0f35e60`
- kind: `fast_withdrawal_service_provider`
- status: `CREATED`
- attempts: xxxxxxxxxxxxx
Found withdrawal, id: `19`
Elapsed time (seconds): `141.9`


- [x] Verify that the user's withdrawal status is updated to `FINISHED` and associated with kind `fast_withdrawal_paid_out`  
- [x] Verify that the withdrawal is considered completed  
- [x] Verify that the withdrawal is considered successful

In [6]:
bridge_operation = await request_bridge_operation_with_high_verbosity(
    test_env=test_env,
    l2_hash=l2_hash,
    kind='fast_withdrawal_payed_out',
    status='FINISHED',
)

l2_transaction = bridge_operation['withdrawal']['l2_transaction']
assert l2_transaction['amount'] == str(full_amount) + '0'*12
assert l2_transaction['l1_account'] == test_env.withdrawer_pkh
assert l2_transaction['l2_account'] == test_env.l2_caller.address.lower().split('0x')[1]
assert bridge_operation['is_completed']
assert bridge_operation['is_successful']
assert bridge_operation['l1_account'] == test_env.withdrawer_pkh
assert bridge_operation['l2_account'] == test_env.l2_caller.address.lower().split('0x')[1]
assert bridge_operation['withdrawal']['l1_transaction'] is not None
assert bridge_operation['withdrawal']['outbox_message']['builder'] == 'service_provider'

l1_transaction = bridge_operation['withdrawal']['l1_transaction']
assert int(l1_transaction['amount']) == int(full_amount * test_env.discount_rate)
assert l1_transaction['sender'] == get_address(test_env.provider)

Requesting bridge operation:
- l2_hash: `be9816c5b30946c73d5a62ef7a78c36940120e90b9d8b47c4613ba4dc0f35e60`
- kind: `fast_withdrawal_payed_out`
- status: `FINISHED`
- attempts: x
Found withdrawal, id: `19`
Elapsed time (seconds): `0.2`


### Test: Indexing `settle_withdrawal` for a withdrawal paid out by the provider  
- [x] Wait for the `SEALED` status of the `fast_withdrawal_service_provider` transaction and call outbox execution

In [7]:
bridge_operation = await request_bridge_operation_with_high_verbosity(
    test_env=test_env,
    l2_hash=l2_hash,
    kind='fast_withdrawal_service_provider',
    status='SEALED',
)

withdrawal = create_withdrawal_from_l2_transaction(l2_transaction)
outbox_message = bridge_operation['withdrawal']['outbox_message']
l1_hash = test_env.make_withdrawal_settlement(outbox_message)

Requesting bridge operation:
- l2_hash: `be9816c5b30946c73d5a62ef7a78c36940120e90b9d8b47c4613ba4dc0f35e60`
- kind: `fast_withdrawal_service_provider`
- status: `SEALED`
- attempts: xxxxxxxxxxxxxxx
Found withdrawal, id: `19`
Elapsed time (seconds): `278.5`
Executing outbox message:
  - Commitment: `src13kJZTJ6kDknT8FpfdxdH2GKyUQTsHbdf4zwhqDQc6xbfymguMY`
  - Proof: `030002fbf1356f4dce4d893721344458534ba6ef1ce70cb5dc6ee96c5d32448c9840a5fbf1356f4dce4d893721344458534ba6ef1ce70cb5dc6ee96c5d32448c9840a50005820764757261626c65d0bf106919ddc2798a9172c550d701c26edaaf399441e3b00e9913557b2213f44903746167c00800000004536f6d650003c08b6f697f6032e3dc255c6cc971d661d6da12202dca93605fcce184a1777dec8a820576616c7565810370766d8107627566666572738205696e707574820468656164c00100066c656e677468c00100066f75747075740004820132810a6c6173745f6c6576656cc0040030dc4f0133810f76616c69646974795f706572696f64c0040003b10082013181086f7574626f78657301bd64015ddc012ef3c0619b36d7f1c9eda7fe89fdef9b856113023572e6df54c7d4e31ab18de127c1d

- [x] Verify that the provider's corresponding record status is updated to `FINISHED`

In [8]:
bridge_operation = await request_bridge_operation_with_high_verbosity(
    test_env=test_env,
    l2_hash=l2_hash,
    kind='fast_withdrawal_service_provider',
    status='FINISHED',
)

l2_transaction = bridge_operation['withdrawal']['l2_transaction']
assert l2_transaction['amount'] == str(full_amount) + '0'*12
assert l2_transaction['l1_account'] == test_env.withdrawer_pkh
assert l2_transaction['l2_account'] == test_env.l2_caller.address.lower().split('0x')[1]
assert bridge_operation['is_completed']
assert bridge_operation['is_successful']
assert bridge_operation['l1_account'] == get_address(test_env.provider)
assert bridge_operation['l2_account'] == test_env.l2_caller.address.lower().split('0x')[1]
assert bridge_operation['withdrawal']['l1_transaction'] is not None
assert bridge_operation['withdrawal']['outbox_message'] is not None

l1_transaction = bridge_operation['withdrawal']['l1_transaction']
# TODO: The L1 transaction amount is None, seems to be `Tezos.amount`?
# assert int(l1_transaction['amount']) == full_amount
assert l1_transaction['sender'] == get_address(test_env.provider)

Requesting bridge operation:
- l2_hash: `be9816c5b30946c73d5a62ef7a78c36940120e90b9d8b47c4613ba4dc0f35e60`
- kind: `fast_withdrawal_service_provider`
- status: `FINISHED`
- attempts: xx
Found withdrawal, id: `19`
Elapsed time (seconds): `1.4`


### Test: Indexing `settle_withdrawal` for a withdrawal not paid out by the provider  
- [ ] Verify that the original withdrawal record with kind `fast_withdrawal` has its status updated to `FINISHED`

### Test: Indexing of `payout_withdrawal` not matching a real withdrawal  
- [ ] Verify that a `payout_withdrawal` event that does not correspond to an actual user withdrawal is still indexed

### Test: `payout_withdrawal` with a different provider recipient address  
- [ ] Verify that the indexer captures information about both the sender and the specified (different) recipient

### Test: Expired `payout_withdrawal` execution  
- [ ] Verify that the service provider receives an indexer entry with status `CREATED` and kind `fast_withdrawal_service_provider`  
- [ ] Verify that the user's withdrawal status is updated to `FINISHED` and associated with kind `fast_withdrawal_paid_out`  
- [ ] Measure the time between the payout's creation on `L1` and the moment it is indexed

In [10]:
# TODO: Wait until the first withdrawal has expired

### Test: Withdrawal to an unconfigured Fast Withdrawal contract with an arbitrary payload, no payouts, outbox executed  
- [ ] Expected behavior check: verify how the underlying outbox message execution is indexed  
- [ ] TODO: Prepare the withdrawal in advance

### Test: Withdrawal to a configured Fast Withdrawal contract with an incorrect payload, no payouts, outbox executed  
- [ ] Verify whether this is indexed similarly to a standard, valid withdrawal  
- [ ] TODO: Prepare the withdrawal in advance

### Test: L1 `payout_withdrawal` indexed before the corresponding L2 withdrawal  
- [ ] Check how the system handles potential indexing desynchronization or race conditions

### Test: Fast Withdrawal settlement of an FA ticket, no payout  
- [ ] Require `fa_withdrawal_precompile` to create an FA Fast Withdrawal